In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('E:/Test/data-analyst-challenge-master')

In [51]:
df = pd.read_json('orders_mini.json')
df.head()

In [52]:
df.rename(columns={'id':'data_id'}, inplace=True)
df['created_at'] = df['created_at'].dt.date
df_1 = df[['data_id','customer','total_price','created_at']]

,id,customer,total_price,created_at,line_items
0,8369263756632563,"{'id': 47178, 'name': 'William Doe', 'email': ...",79.39,2020-03-07 14:31:11+00:00,"[{'id': 610448, 'product_id': 632910392, 'prod..."
1,7262170348080494,"{'id': 94720, 'name': 'Emile Tumson', 'email':...",86.98,2020-03-01 09:16:30+00:00,"[{'id': 997208, 'product_id': 278266679, 'prod..."
2,7722055557038194,"{'id': 59933, 'name': 'Ethan Jones', 'email': ...",36.99,2020-01-01 09:17:03+00:00,"[{'id': 759658, 'product_id': 573794972, 'prod..."
3,4280852170163518,"{'id': 23955, 'name': 'Daniel Smith', 'email':...",55.98,2020-03-10 11:59:46+00:00,"[{'id': 317808, 'product_id': 477365777, 'prod..."
4,5058732129957127,"{'id': 59933, 'name': 'Ethan Jones', 'email': ...",309.98,2020-03-15 07:27:24+00:00,"[{'id': 604665, 'product_id': 510650809, 'prod..."


In [53]:
df_1 = pd.concat([df_1.drop(['customer'], axis=1), df_1['customer'].apply(pd.Series)], axis=1)
df_2 = df[['data_id','line_items']]
df_2 = pd.concat([df_2.drop(['line_items'], axis=1)
                , df_2['line_items'].apply(lambda x:pd.Series(x[0] if len(x)>0 else {}))], axis=1)
df_2.rename(columns={'id':'order_id'}, inplace=True)
df_new = pd.concat([df_1, df_2], axis=1)
df_new = df_new[['data_id','total_price','id','name','email','created_at','order_id','product_id','product_sku','product_name','price']]

In [68]:
import requests
url = 'https://api.exchangeratesapi.io/history?start_at=2020-01-01&end_at=2020-04-01&base=USD'
response = requests.get(url)
data = response.json()
df_curr = pd.DataFrame(data)
df_curr.head()

In [74]:
df_curr['Order_Date'] = df_curr.index
df_curr_1 = df_curr[['rates','base','Order_Date']]
df_curr_1 = pd.concat([df_curr_1.drop(['rates'], axis=1), df_curr_1['rates'].apply(pd.Series)], axis=1)
df_curr_1 = df_curr_1[['base', 'Order_Date','CAD']]
df_curr_1['Order_Date'] = pd.to_datetime(df_curr_1['Order_Date']).dt.date
df_final = df_new.merge(df_curr_1, left_on='created_at', right_on='Order_Date')
df_final.to_excel('Final_Data.xlsx')